感知器中常用的阶跃函数：

$$
\operatorname{heaviside}(z) =
\begin{cases}
0, & \text{如果 } z < 0 \\
1, & \text{如果 } z \geq 0
\end{cases}
\qquad
\operatorname{sgn}(z) =
\begin{cases}
-1, & \text{如果 } z < 0 \\
0,  & \text{如果 } z = 0 \\
+1, & \text{如果 } z > 0
\end{cases}
$$

单个TLU可用于简单的线性二进制分类。它计算输入的线性组合，如果结果超过阈值，则输出正类；否则，它将输出负类。

感知器仅由单层TLU组成，每个TLU连接到所有的输入。当一层中的所有神经元都连接到上一层中的每个神经元（即其输入神经元）时，该层称为全连接层或密集层。感知器的输入被送到称为输入神经元的特殊直通神经元：它们输出被送入的任何输入。所有输入神经元形成输入层。此外，通常会添加一个额外的偏置特征（x0=1）：通常使用一种称为偏置神经元的特殊类型的神经元来表示该特征，该神经元始终输出1。

计算全连接层的输出：

$h_{W,b}(X)=\phi (XW+b)$

1. $X$代表输入特征的矩阵，每个实例一行，每个特征一列。
2. 权重矩阵$W$包含除偏置神经元外的所有连接权重。在该层中，每个输入神经元一行，每个人工神经元一列。
3. 偏置向量$b$包含偏置神经元和人工神经元之间的所有连接权重。每个人工神经元有一个偏置项。
4. 函数$\phi$称为激活函数：当人工神经元是TLU时，它是阶跃函数

感知器学习规则：感知器一次被送入一个训练实例，并且针对每个实例进行预测。对于产生错误预测的每个输出神经元，它会增强来自输入的连接权重，这些权重将有助于正确的预测。

$w_{i,j}^{(下一步)}=w_{i,j}+\eta(y_j-\hat{y_j})x_i$
1. $w_{i,j}$是第$i$个输入神经元和第$j$个输出神经元之间的连接权重。
2. $x_i$是当前训练实例的第$i$个输入值。
3. $\hat{y_j}$是当前训练实例的第$j$个输出神经元的输出。
4. $y_j$是当前训练实例的第$j$个输出神经元的目标输出。
5. $\eta$是学习率。

In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron
iris  = load_iris()
X = iris.data[:, (2,3)]
y = (iris.target == 0).astype(int)

per_clf = Perceptron()
per_clf.fit(X, y)

y_pred = per_clf.predict([[2, 0.5]])

自动计算梯度算法（autodiff）：
1. 它一次处理一个小批量，并且多次遍历整个训练集。每次遍历都称为一个轮次。
2. 每个小批量都传递到网络的输入层，然后将其送到第一个隐藏层。然后该算法将计算该层中所有神经元的输出（对于小批量中的每个实例）。结果传递到下一层，计算其输出并传递到下一层，以此类推，直到获得最后一层（即输出层）的输出。这就是前向通路：就像进行预测一样，只是保留了所有中间结果，因为反向遍历需要它们。
3. 接下来，该算法测量网络的输出误差（该算法使用一种损失函数，该函数将网络的期望输出与实际输出进行比较，并返回一些误差测量值）。
4. 然后，它计算每个输出连接对错误的贡献程度。通过应用链式法则来进行分析，从而使此步骤变得快速而精确
5. 然后，算法再次使用链式法则来测量这些错误贡献中有多少是来自下面层中每个连接的错误贡献，算法一直进行，到达输入层为止。如前所述，这种反向传递通过在网络中向后传播误差梯度，从而有效地测量了网络中所有连接权重上的误差梯度。
6. 最终，该算法执行梯度下降步骤，使用刚刚计算出的误差梯度来调整网络中的所有连接权重。

常用激活函数：

逻辑函数：$\sigma(z)=\frac{1}{1+e^{-z}}\ \ \ \ $各处均有非零导数

双曲正切函数：$tanh(z)=2\sigma(2z)-1\ \ \ \ $有助于加快收敛速度

线性整流单位函数：$ReLU(z)=max(0,z)\ \ \ \ $运行良好且计算快速

MLP：多层感知器

MLP由一层（直通）输入层、一层或多层TLU（称为隐藏层）和一个TLU的最后一层（称为输出层）组成。靠近输入层的层通常称为较低层，靠近输出层的层通常称为较高层。除输出层外的每一层都包含一个偏置神经元，并完全连接到下一层。

回归MLP：

每个输出维度需要一个输出神经元。可以使用softplus激活函数：$softplus(z)=log(1+e^{z})$（正输出）。若要保证预测值落在给定的值范围内，则可以使用逻辑函数或双曲正切（有界输出），然后将标签缩放到适当的范围。损失函数通常使用均方误差，如果训练集中有很多离群值，则可使用平均绝对误差。或者，可以使用Huber损失，这是两者的组合。（当误差小于阈值δ（通常为1）时，Huber损失为二次方，而当误差大于δ时，Huber损失为线性。线性部分使它对离群值的敏感性低于均方误差，而二次方部分使它比平均绝对误差更收敛并且更精确。）

分类MLP：
1. 对于二进制分类问题，你只需要使用逻辑激活函数的单个输出神经元：输出将是0到1之间的数字，你可以将其解释为正类的估计概率。负类别的估计概率等于一减去该数字。
2. 多标签二进制分类任务，为每个正类别用一个输出神经元，输出概率不一定要加起来为1，这可以使模型输出任何组合的标签
3. 如果每个实例只能属于三个或更多可能的类中的一个类，则每个类需要一个输出神经元，并且应该使用softmax激活函数整个输出层，确保所有估计的概率在0到1之间，并且它们加起来等于1，称为多类分类。
损失函数使用交叉熵损失